In [115]:
import pandas as pd
import plotly.express as px
from sklearn.utils import shuffle
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

def sampling(df):
    return df.head(10)

pd.options.mode.chained_assignment = None  # default='warn'

dataset = pd.read_csv('msd_genre_dataset0.csv', header=0)

classicPopAndRock = dataset.iloc[0:23895]
punk = dataset.iloc[23895:27095]
folk = dataset.iloc[27095:40287]
pop = dataset.iloc[40287:41904]
danceAndElectronica = dataset.iloc[41904:46839]
metal = dataset.iloc[46839:48942]
jazz = dataset.iloc[48942:53276]
classical = dataset.iloc[53276:55150]
hipHop = dataset.iloc[55150:55584]
soulAndReggae = dataset.iloc[55584:59600]

# classicPopAndRock.name = 'classicPopAndRock'
# punk.name = 'punk'
# folk.name = 'folk'
# pop.name = 'pop'
# danceAndElectronica.name = 'danceAndElectronica'
# jazz.name = 'jazz'
# classical.name = 'classical'
# hipHop.name = 'hipHop'
# soulAndReggae.name = 'soulAndReggae'

genre = ['classicPopAndRock', 'punk', 'folk', 'pop', 'danceAndElectronica', 'metal',
         'jazz', 'classical', 'hipHop', 'soulAndReggae']
timbreFeaturesAndGenre = ['avg_timbre1','avg_timbre2','avg_timbre3','avg_timbre4', 'avg_timbre5',
                'avg_timbre6','avg_timbre7','avg_timbre8', 'avg_timbre9','avg_timbre10',
                'avg_timbre11','avg_timbre12', 'var_timbre1', 'var_timbre2', 'var_timbre3',
                'var_timbre4', 'var_timbre5','var_timbre6','var_timbre7','var_timbre8',
                'var_timbre9','var_timbre10','var_timbre11','var_timbre12', 'genre']

datasetFrames = [sampling(classicPopAndRock),sampling(punk), sampling(folk),sampling(pop),
                 sampling(danceAndElectronica), sampling(metal), sampling(jazz),
                 sampling(classical), sampling(hipHop), sampling(soulAndReggae)]

datasetSample = pd.concat(datasetFrames)
shuffledDatasetSample = shuffle(datasetSample)
featureVector = suffledDatasetSample[timbreFeaturesAndGenre]

kMeans = KMeans(n_clusters=10)
timbreFeatures = timbreFeaturesAndGenre[:-1]
kMeans.fit(featureVector[timbreFeatures])
testingSet = featureVector
featureVector.loc[:, 'genre'] = kMeans.labels_

genreNumbers = featureVector[['genre']]
genreTitles = shuffledDatasetSample[['genre']]
genreNumbers['genre'] = genreNumbers['genre'].astype(str)

newDf = pd.DataFrame()
newDf['genreN'] = genreNumbers['genre']
newDf['genreT'] = genreTitles['genre']

for index, row in newDf.iterrows():
    print(row['genreN'], row['genreT'])

confusionMatrix = pd.DataFrame(confusion_matrix(featureVector[['genre']], testingSet[['genre']]),
                       index=genre,
                       columns=genre)

#print(confusionMatrix)

# punkSong = punk.iloc[11]
# punkSongTimbre = punkSong[timbreFeaturesAndGenre]
# noGenre = punkSongTimbre[:-1]
# print(noGenre)
# [noGenre[i:i+1] for i in range(0, len(noGenre), 1)]
# #noGenre.reshape(-1,1)
# print(noGenre)

# kMeans.predict(noGenre)

5 classic pop and rock
2 soul and reggae
2 soul and reggae
1 jazz and blues
0 metal
0 punk
6 classical
0 punk
1 metal
9 pop
1 classic pop and rock
5 folk
0 classic pop and rock
6 classical
2 soul and reggae
0 metal
4 dance and electronica
8 jazz and blues
2 soul and reggae
0 classic pop and rock
5 folk
3 dance and electronica
0 classic pop and rock
2 folk
0 punk
9 punk
9 folk
6 hip-hop
5 metal
5 pop
7 classic pop and rock
5 folk
1 classical
2 soul and reggae
6 classical
5 jazz and blues
6 classical
9 dance and electronica
0 punk
5 folk
0 classic pop and rock
6 hip-hop
5 pop
9 hip-hop
6 classical
2 soul and reggae
6 hip-hop
8 jazz and blues
5 pop
9 pop
9 hip-hop
0 classic pop and rock
5 hip-hop
0 metal
0 punk
0 hip-hop
5 folk
5 pop
2 soul and reggae
2 soul and reggae
6 classical
8 dance and electronica
6 classical
0 hip-hop
8 dance and electronica
0 classic pop and rock
5 punk
2 soul and reggae
0 metal
6 classical
0 classic pop and rock
5 punk
1 folk
5 jazz and blues
5 hip-hop
7 dance a

In [ ]:
# def extractTimbreMeans(df):
#     avgTimbreDf = df[['avg_timbre1','avg_timbre2','avg_timbre3','avg_timbre4',
#                       'avg_timbre5','avg_timbre6','avg_timbre7','avg_timbre8',
#                       'avg_timbre9','avg_timbre10','avg_timbre11','avg_timbre12']]
#     df['avgTimbreMean'] = avgTimbreDf.sum(axis=1)
#     varTimbreDf = df[['var_timbre1','var_timbre2','var_timbre3','var_timbre4',
#                       'var_timbre5','var_timbre6','var_timbre7','var_timbre8',
#                       'var_timbre9','var_timbre10','var_timbre11','var_timbre12']]
#     df['varTimbreMean'] = varTimbreDf.sum(axis=1)
#     return df

# def extractVarTimbre(df):
#     varTimbreDf = df[['var_timbre1','var_timbre2','var_timbre3','var_timbre4',
#                       'var_timbre5','var_timbre6','var_timbre7','var_timbre8',
#                       'var_timbre9','var_timbre10','var_timbre11','var_timbre12']]
#     df['mean'] = varTimbreDf.mean(axis=1)
#     return df

# classicPopAndRockAvgTimbre = extractAvgTimbre(classicPopAndRock)
# punkAvgTimbre = extractAvgTimbre(punk)
# folkAvgTimbre = extractAvgTimbre(folk)
# popAvgTimbre = extractAvgTimbre(classicPopAndRock)
# danceAndElectronicaAvgTimbre = extractAvgTimbre(danceAndElectronica)
# metalAvgTimbre = extractAvgTimbre(metal)
# jazzAvgTimbre = extractAvgTimbre(jazz)
# classicalAvgTimbre = extractAvgTimbre(classical)
# hipHopAvgTimbre = extractAvgTimbre(hipHop)
# soulAndReggaeAvgTimbre = extractAvgTimbre(soulAndReggae)

# classicPopAndRockVarTimbre = extractAvgTimbre(classicPopAndRock)
# punkVarTimbre = extractVarTimbre(punk)
# folkVarTimbre = extractVarTimbre(folk)
# popVarTimbre = extractVarTimbre(classicPopAndRock)
# danceAndElectronicaVarTimbre = extractVarTimbre(danceAndElectronica)
# metalVarTimbre = extractVarTimbre(metal)
# jazzVarTimbre = extractVarTimbre(jazz)
# classicalVarTimbre = extractVarTimbre(classical)
# hipHopVarTimbre = extractVarTimbre(hipHop)
# soulAndReggaeVarTimbre = extractVarTimbre(soulAndReggae)

# fig = plt.figure(figsize=(12, 9))
# ax = Axes3D(fig, elev=21, azim=-136)
# ax.scatter(dataframe['Top_Tail'], dataframe['Middle_Segment'], dataframe['Bottom_Tail'], 
#            c=labels.astype(np.float))
# ax.set_xlabel('TopTail')
# ax.set_ylabel('MiddleSegment')
# ax.set_zlabel('BottomTail')
# plt.show()

# fig = px.scatter(avgTimbreSample, x=avgTimbre1, y=avgTimbre2, color='genre')
# fig.show()

# fig = px.box(avgTimbreSample, x="genre", y=avgTimbre1)
# fig.show()